In [1]:
from openvino_tokenizers import convert_tokenizer
import openvino as ov

from transformers import AutoTokenizer
import numpy as np

core = ov.Core()

model_id = "microsoft/phi-1_5"
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model_id = "katuni4ka/tiny-random-llava-next"
model_id = "answerdotai/ModernBERT-base"

hf_tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# hf_tokenizer.save_pretrained(f'{model_id.split("/")[1]}')
ov_tokenizer, ov_detokenizer = convert_tokenizer(hf_tokenizer, with_detokenizer=True, number_of_inputs=2, max_length=20)
# ov_tokenizer, ov_detokenizer = convert_tokenizer(hf_tokenizer, with_detokenizer=True)

ov.save_model(ov_tokenizer, f'{model_id.split("/")[1]}/openvino_tokenizer.xml')
ov.save_model(ov_detokenizer, f'{model_id.split("/")[1]}/openvino_detokenizer.xml')

/Users/pavel-esir/opt/envs/py312-ov-tokenizers/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/pavel-esir/opt/openvino_master/python/openvino/runtime/__init__.py:10: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  warnings.warn(


In [2]:
tokenizer = core.compile_model(ov_tokenizer, "CPU")
detokenizer = core.compile_model(ov_detokenizer, "CPU")

In [3]:
# # ov_tokenizer.reshape(dict(Parameter_1=[2, 1], Parameter_2=[2, 1]))
# ov_tokenizer.reshape(ov.PartialShape([2]))
# ov.serialize(ov_tokenizer, "tok.xml")
# tokenizer = core.compile_model(ov_tokenizer, "CPU")

# from openvino import passes
# pass_manager = passes.Manager()
# pass_manager.register_pass(passes.VisualizeTree(file_name='image.svg'))
# pass_manager.run_passes(ov_tokenizer)

# inp_1 = ov.Tensor(["hi"])
# inp_2 = ov.Tensor(["sun in yellow"])
# # inp_2.set_shape([0])
# tokenizer([[inp_1, inp_2]])

In [4]:
hf_tokenizer([["hi", "sun in yellow"]], return_tensors="np")

{'input_ids': array([[50281,  5801, 50282, 13998,   275,  8862, 50282]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1]])}

In [5]:
tokenizer([["hi", "sun in yellow"]])
# [1, 2] for solution with reshape split

# tokenizer([["hi"], ["sun in yellow"]])
# [2, 1] for solution with concat split

{<ConstOutput: names[input_ids] shape[?,?] type: i64>: array([[50281,  5801, 50282, 13998,   275,  8862, 50282]]), <ConstOutput: names[attention_mask] shape[?,?] type: i64>: array([[1, 1, 1, 1, 1, 1, 1]])}

In [6]:
# tokenizer(["hi", "sun in yellow sdsf sdfsd 8892 sdfsk dssr 9sd 1k sd"])[0][0].shape

In [7]:
tokenizer([["hi"]])

{<ConstOutput: names[input_ids] shape[?,?] type: i64>: array([[50281,  5801, 50282]]), <ConstOutput: names[attention_mask] shape[?,?] type: i64>: array([[1, 1, 1]])}

In [22]:
import random
Nx = 140
Ny = 10
get_seg = lambda N: ' '.join([chr(random.randint(0, 127)) for i in range(N)])
x = get_seg(Nx)
y = get_seg(Ny)

In [23]:
len(x), len(y)

(279, 19)

In [24]:
res_hf = hf_tokenizer([[x, y]], max_length=20, truncation=True, return_tensors="np")['input_ids']
res_ov = tokenizer([[x, y]])[0]
res = res_hf

print(res)
print('------'*10)

idx = np.nonzero(res[0] == 50282)[0][0]
print(idx)

[[50281    74   209   194   247   500  3706  5042   416   259 50282   193
    353   416   258   209   183   270 50275 50282]]
------------------------------------------------------------
10


In [25]:
res_hf

array([[50281,    74,   209,   194,   247,   500,  3706,  5042,   416,
          259, 50282,   193,   353,   416,   258,   209,   183,   270,
        50275, 50282]])

In [26]:
res_ov

array([[50281,    74,   209,   194,   247,   500,  3706,  5042,   416,
          259, 50282,   193,   353,   416,   258,   209,   183,   270,
        50275, 50282]])

In [13]:
hf_tokenizer.decode(res_hf[0])

'[CLS]\x07 : k @ ( P x + q [ ] *, Y @ k[SEP]\x1b[SEP]'

In [14]:
hf_tokenizer.decode(res_ov[0])

'[CLS]\x07 : k @ ( P x + q[SEP]\x1b|||IP_ADDRESS||||||IP_ADDRESS||||||IP_ADDRESS|||<|padding|>|||IP_ADDRESS||||||IP_ADDRESS|||[SEP]'

In [15]:
len(res[0])

20

In [16]:
# print(x)
print(y)




In [17]:
decoded = hf_tokenizer.decode(res[0])
print(decoded)

[CLS] : k @ ( P x + q [ ] *, Y @ k[SEP]EP]


In [18]:
decoded.count('s')

0

In [19]:
decoded.count('h')

0

In [20]:
hf_tokenizer.encode('[SEP]', add_special_tokens=False)

[50282]

Single string can occupy more than 50% only if the second one is shorter than that.

if sum is less than max length then no truncation.
Select (sum <= max_length - num_added_tokens, x, slice(x, 0, len(x) // 2))

opset.shape_of(x)
opset.less(input_nodes[0], max_length).output(0),

If sum > max_length

x -> x[:len(x) // 2]
y -> y[:len(y) // 2]

Select
